In [4]:
import re
"""
# 部署名フォーマット処理設計書

## 1. 概要
本システムは、部署名を入力として受け取り、特定のルールに基づいてフォーマットを行う機能を提供します。

## 2. 機能要件
- 個別要件に合致する部署名はそのまま返す
- 全体マッチの例外ケースに合致する部署名はそのまま返す
- 「〜部(拠点名)」パターンの部署名はそのまま返す
- その他の部署名に対して、「部」の後にスペースを挿入する
- 特定のパターンに対してはスペース挿入をキャンセルする

## 3. 入力
- 部署名（文字列）

## 4. 出力
- フォーマット済み部署名（文字列）

## 5. 処理フロー
以下にPlantUMLによる処理フローを示します：

```plantuml
@startuml
start
:部署名を入力;
if (個別要件リストに一致?) then (yes)
    :部署名をそのまま返す;
    stop
endif
if (全体マッチ例外リストに一致?) then (yes)
    :部署名をそのまま返す;
    stop
endif
if (「〜部(拠点名)」パターンに一致?) then (yes)
    :部署名をそのまま返す;
    stop
endif
:括弧内外で分割;
while (未処理の分割部分がある?) is (yes)
    :「部」の後にスペースを挿入;
    :特定パターンのスペースを削除;
endwhile
:分割部分を結合;
:部分マッチ例外処理;
:結果を返す;
stop
@enduml
"""
def format_department_name(department_name):
    # 個別要件リスト（完全一致でバイパス）
    individual_requirements = [
        "営業本部業務部",
        "内部監査部業務監査グループ第三ユニット",
    ]

    # 全体マッチの例外ケース（完全一致でバイパス）
    full_match_exceptions = [
        "本部審議役",
        "本部参事役",
    ]

    # 部分マッチの例外ケース（部分一致で処理）
    partial_match_exceptions = [
        "中部",
        "春日部",
        "宇部",
        "東部",
        "西部",
    ]

    # 個別要件チェック
    if department_name in individual_requirements:
        return department_name

    # 全体マッチの例外チェック
    if department_name in full_match_exceptions:
        return department_name

    # 「〜部(拠点名)」パターンのチェック
    if re.search(r'部\([^)]*\)$', department_name):
        return department_name

    # 括弧内の処理を避けるため、括弧内外で分割
    parts = re.split(r'(\([^)]*\))', department_name)
    
    # 部店名の()出現パターンに依存した書き方になっているので留意
    # 括弧の外側の部分のみ処理
    for i in range(0, len(parts), 2):
        # 「部」の後にスペースを挿入（最後の「部」を除く）
        # 否定先読み?!を適用しカッコ内のパターンに一致しない場合のみ
        # その位置でマッチする
        parts[i] = re.sub(r'部(?!$)', '部 ', parts[i])

        # 一旦、部キーワードで半角ブランク挿入
        #if not parts[i].endswith('部'):  # 「部」で終わる場合はスペースを追加しない
        #    parts[i] = parts[i].replace("部", "部 ")

        # ブランク戻しパターン
        parts[i] = parts[i].replace("部 部 ", "部部 ")
        parts[i] = parts[i].replace("部 門", "部門")
        parts[i] = parts[i].replace("部部門", "部 部門")
        parts[i] = parts[i].replace("部門 (", "部門(")
        parts[i] = parts[i].replace("部 ·", "部·")

    department_name = ''.join(parts)

    # 部分マッチの例外処理（ブランク戻し）
    for exception_case in partial_match_exceptions:
        if exception_case in department_name:
            department_name = department_name.replace(exception_case + " ", exception_case)

    return department_name.strip()

# テストケース
test_cases = [
    "さいたま本部春日部部クレヨン課",
    "営業本部業務部",
    "内部監査部業務監査グループ第三ユニット",
    "トランザクション監視部門(東京本部)",
    "aaa本部bbbb部xxxx課",
    "中部春日部東部西部",
    "システム統括部システム企画課",
    "法人・リテールリスク統括部部門検査室",
    "園田寮(69101)人事部·大",
    "天満研修所(69101)人事部·大",
    "浜甲子園寮(69101)人事部·大",
    "茨木西寮(69101)人事部·大",
    "京町堀5(69101)人事部·大",
    "総務部総務課",
    "本部審議役",
    "トランザクション監視部門(天津)",
    "部門検査室",
    "部門検査室(大阪)",
    "〜部(拠点名)",
    "テスト部部テスト課",
    "テスト部門",
    "リテールリスク統括部部門管理部",
    "営業本部ナゴヤ営業第一本部",
    "営業本部ナゴヤ営業第一本部第一Gr",
]

# テストの実行
for case in test_cases:
    result = format_department_name(case)
    print(f"Original: {case}")
    print(f"Processed: {result}")
    print()

Original: さいたま本部春日部部クレヨン課
Processed: さいたま本部 春日部部 クレヨン課

Original: 営業本部業務部
Processed: 営業本部業務部

Original: 内部監査部業務監査グループ第三ユニット
Processed: 内部監査部業務監査グループ第三ユニット

Original: トランザクション監視部門(東京本部)
Processed: トランザクション監視部門(東京本部)

Original: aaa本部bbbb部xxxx課
Processed: aaa本部 bbbb部 xxxx課

Original: 中部春日部東部西部
Processed: 中部春日部東部西部

Original: システム統括部システム企画課
Processed: システム統括部 システム企画課

Original: 法人・リテールリスク統括部部門検査室
Processed: 法人・リテールリスク統括部 部門検査室

Original: 園田寮(69101)人事部·大
Processed: 園田寮(69101)人事部·大

Original: 天満研修所(69101)人事部·大
Processed: 天満研修所(69101)人事部·大

Original: 浜甲子園寮(69101)人事部·大
Processed: 浜甲子園寮(69101)人事部·大

Original: 茨木西寮(69101)人事部·大
Processed: 茨木西寮(69101)人事部·大

Original: 京町堀5(69101)人事部·大
Processed: 京町堀5(69101)人事部·大

Original: 総務部総務課
Processed: 総務部 総務課

Original: 本部審議役
Processed: 本部審議役

Original: トランザクション監視部門(天津)
Processed: トランザクション監視部門(天津)

Original: 部門検査室
Processed: 部門検査室

Original: 部門検査室(大阪)
Processed: 部門検査室(大阪)

Original: 〜部(拠点名)
Processed: 〜部(拠点名)

Original: テスト部部テスト課
Processed: テスト部部 テスト課

Origin